In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from wrangle import *

from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from math import sqrt

from scipy import stats
from pydataset import data

In [3]:
def scale_data(df):
    """
    Accepts a dataframe and scales the data using MinMax
    """
    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    return scaled_data

In [4]:
df = wrangle_zillow()

/Users/dusts/codeup-data-science/zillow-project/wrangle.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fips'] = df['fips'].apply(int).copy()
/Users/dusts/codeup-data-science/zillow-project/wrangle.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fips'] = df['fips'].apply(str).copy()
/Users/dusts/codeup-data-science/zillow-project/wrangle.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [5]:
df.head()

,parcelid,id,bathroom_count,bedroom_count,calculated_finished_square_feet,finished_square_feet_12,fips,full_bath_count,latitude,longitude,...,structure_taxvalue_dollarcount,tax_valuedollar_count,assessment_year,land_tax_value_dollar_count,tax_amount,census_tract_and_block,id_1,log_error,transaction_date,bed_bath_count
0,14297519,1727539,3.5,4.0,3100.0,3100.0,06059,3.0,33634931.0,-117869207.0,...,485713.0,1023282.0,2016,537569.0,11013.72,6.059063e+13,0,0.025595,2017-01-01,7.5
1,17052889,1387261,1.0,2.0,1465.0,1465.0,06111,1.0,34449266.0,-119281531.0,...,88000.0,464000.0,2016,376000.0,5672.48,6.111001e+13,1,0.055619,2017-01-01,3.0
2,14186244,11677,2.0,3.0,1243.0,1243.0,06059,2.0,33886168.0,-117823170.0,...,85289.0,564778.0,2016,479489.0,6488.30,6.059022e+13,2,0.005383,2017-01-01,5.0
3,12177905,2288172,3.0,4.0,2376.0,2376.0,06037,3.0,34245180.0,-118240722.0,...,108918.0,145143.0,2016,36225.0,1777.51,6.037300e+13,3,-0.103410,2017-01-01,7.0
4,12095076,781532,3.0,4.0,2962.0,2962.0,06037,3.0,34145202.0,-118179824.0,...,276684.0,773303.0,2016,496619.0,9516.26,6.037461e+13,6,-0.001011,2017-01-01,7.0


In [6]:
def separate_columns(df):
  continuous_columns = []
  discrete_columns = []

  for column in df.columns:
    if df[column].dtype == 'int' or df[column].dtype == 'float':
      continuous_columns.append(column)
    else:
      discrete_columns.append(column)

  df_continuous = df[continuous_columns]
  df_discrete = df[discrete_columns]

  return df_continuous, df_discrete

In [8]:
continous, discrete = separate_columns(df)

In [10]:
continous.head()

,parcelid,id,bathroom_count,bedroom_count,calculated_finished_square_feet,finished_square_feet_12,full_bath_count,latitude,longitude,lot_size_square_feet,...,year_built,structure_taxvalue_dollarcount,tax_valuedollar_count,assessment_year,land_tax_value_dollar_count,tax_amount,census_tract_and_block,id_1,log_error,bed_bath_count
0,14297519,1727539,3.5,4.0,3100.0,3100.0,3.0,33634931.0,-117869207.0,4506.0,...,1998,485713.0,1023282.0,2016,537569.0,11013.72,6.059063e+13,0,0.025595,7.5
1,17052889,1387261,1.0,2.0,1465.0,1465.0,1.0,34449266.0,-119281531.0,12647.0,...,1967,88000.0,464000.0,2016,376000.0,5672.48,6.111001e+13,1,0.055619,3.0
2,14186244,11677,2.0,3.0,1243.0,1243.0,2.0,33886168.0,-117823170.0,8432.0,...,1962,85289.0,564778.0,2016,479489.0,6488.30,6.059022e+13,2,0.005383,5.0
3,12177905,2288172,3.0,4.0,2376.0,2376.0,3.0,34245180.0,-118240722.0,13038.0,...,1970,108918.0,145143.0,2016,36225.0,1777.51,6.037300e+13,3,-0.103410,7.0
4,12095076,781532,3.0,4.0,2962.0,2962.0,3.0,34145202.0,-118179824.0,63000.0,...,1950,276684.0,773303.0,2016,496619.0,9516.26,6.037461e+13,6,-0.001011,7.0
